In [3]:
#Importing required libraries
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from imblearn.datasets import make_imbalance
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, cross_val_score
from sklearn.utils import resample,shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
import random
import os

In [4]:
path = '/Users/kiranjyothisheena/Documents/Kiran_Files/Cloud_computing/Project/'
df = pd.read_excel(path + 'Telecom Churn Rate Dataset.xlsx')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56.95,1889.5,0,0,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0,0,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,0,0,Yes


In [5]:
#Class imbalance
df['Churn'].value_counts(normalize=True)

No     0.73463
Yes    0.26537
Name: Churn, dtype: float64

# Data Cleaning

In [6]:
df_2 = df.copy()
df_2['TotalCharges'] = df_2['TotalCharges'].replace(' ',np.nan)
df_2 = df_2.dropna(how='any').reset_index(drop=True)
df_2

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56.95,1889.50,0,0,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0,0,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,0,0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,0,0,No
7028,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,0,5,No
7029,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,0,No
7030,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,0,0,Yes


In [7]:
from sklearn.preprocessing import OneHotEncoder
def get_ohe(df,col_name):
    ohe = OneHotEncoder(sparse=False,categories="auto",drop="first")
    ohe.fit(df[col_name])
    temp_df = pd.DataFrame(data=ohe.transform(df[col_name]), columns=ohe.get_feature_names_out())
    df.drop(columns=col_name, axis=1, inplace=True)
    df = pd.concat([df.reset_index(drop=True), temp_df], axis=1)
    return df,ohe
df_3,ohe_obj = get_ohe(df_2,["gender","Partner", "Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity",
                            "OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling",
                            "PaymentMethod","Churn"])

/Users/kiranjyothisheena/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
df_3.head()

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,gender_Male,Partner_Yes,Dependents_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,7590-VHVEG,0,1,29.85,29.85,0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,5575-GNVDE,0,34,56.95,1889.50,0,0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3668-QPYBK,0,2,53.85,108.15,0,0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,7795-CFOCW,0,45,42.30,1840.75,0,3,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9237-HQITU,0,2,70.70,151.65,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [9]:
df_4 = df_3.drop(columns = ['customerID'])
df_4.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,0,1,29.85,29.85,0,0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0,34,56.95,1889.50,0,0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,2,53.85,108.15,0,0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0,45,42.30,1840.75,0,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,70.70,151.65,0,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


# Modeling

In [10]:
#Train-test split
train, test = train_test_split(df_4, test_size = 0.2)
print(train.shape)

(5625, 33)


In [11]:
#Upsampling to handle class imbalance
train_minority = train[train['Churn_Yes'] == 1]
train_other = train[train['Churn_Yes'] == 0]

min_upsampled = resample(train_minority,random_state=42,n_samples=len(train_other)-len(train_minority),replace=True)
train_upsampled = pd.concat([min_upsampled,train], axis = 0, ignore_index=True, sort=False)

#Splitting to X, y
train_X = train_upsampled.drop(columns = ['Churn_Yes'])
train_y = train_upsampled['Churn_Yes']

In [12]:
#Making test_X and test_y
test_X = test.drop(columns = ['Churn_Yes'])
test_y = test['Churn_Yes']

In [13]:
#Making folds for CV
#5 fold CV is used since it was taking too long for 10 fold on this data set
folds = KFold(n_splits=10, shuffle = True, random_state = 421)
folds

KFold(n_splits=10, random_state=421, shuffle=True)

### Random Forest 

In [14]:
#Checking test score with best model
best_model = RandomForestClassifier(n_estimators = 500,
                                    max_features = 3, 
                                    min_samples_leaf = 2,
                                    bootstrap = True, 
                                    max_depth = 15).fit(train_X, train_y)
predictions = best_model.predict(test_X)

In [15]:
#Metrics on Test set
print('Test F1 score: ', f1_score(test_y, predictions))
print('Test Accuracy Score: ', accuracy_score(test_y, predictions))

Test F1 score:  0.7225647348951911
Test Accuracy Score:  0.8400852878464818


## EDA on Churn segments

In [16]:
## Obtaining churn probabilities

df_5 = df_4.drop(["Churn_Yes"],axis=1)
pred_prob_overall = best_model.predict_proba(df_5)
df_5['Churn_Prob'] = pred_prob_overall[:,1]

In [17]:
path = '/Users/kiranjyothisheena/Documents/Kiran_Files/Cloud_computing/Project/'
df_6 = pd.read_excel(path + 'Telecom Churn Rate Dataset.xlsx')

df_6['TotalCharges'] = df_6['TotalCharges'].replace(' ',np.nan)
df_6 = df_6.dropna(how='any').reset_index(drop=True)

df_6['Churn_Prob'] = pred_prob_overall[:,1]
df_6.sort_values(by=["Churn_Prob"],inplace=True,ascending=False)
df_6

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn,Churn_Prob
994,1374-DMZUI,Female,1,No,No,4,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,94.30,424.45,1,2,Yes,0.988220
1699,0107-YHINA,Male,0,No,Yes,1,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,99.75,99.75,0,1,Yes,0.987897
966,3158-MOERK,Female,0,No,No,2,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,96.00,174.80,0,1,Yes,0.987731
1971,9497-QCMMS,Male,1,No,No,1,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,93.55,93.55,0,1,Yes,0.987372
842,3027-ZTDHO,Male,0,No,No,1,Yes,No,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,89.90,89.90,0,2,Yes,0.986069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,0971-QIFJK,Female,0,Yes,No,69,Yes,Yes,No,No internet service,...,No internet service,Two year,No,Bank transfer (automatic),24.25,1641.80,0,0,No,0.000412
3977,5884-FBCTL,Female,0,Yes,Yes,72,Yes,Yes,No,No internet service,...,No internet service,Two year,No,Bank transfer (automatic),25.10,1857.85,4,0,No,0.000378
844,6916-HIJSE,Female,0,No,No,65,Yes,No,DSL,Yes,...,Yes,Two year,Yes,Bank transfer (automatic),84.85,5459.20,0,0,No,0.000308
28,5248-YGIJN,Male,0,Yes,No,72,Yes,Yes,DSL,Yes,...,Yes,Two year,Yes,Credit card (automatic),90.25,6369.45,1,0,No,0.000279


In [72]:
## Creating buckets based on churn rate

df_6['Churn_bin'] = pd.cut(df_6['Churn_Prob'],
                           bins=[0, 0.5, 0.7, 0.9 , 1],
                           labels=['low', 'slight',"medium", 'high'],
                          include_lowest=True)
                                  
with pd.option_context("display.max.columns",None):
    display(df_6)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn,Churn_Prob,Churn_bin
1971,9497-QCMMS,Male,1,No,No,1,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,93.55,93.55,0,1,Yes,0.994202,high
6359,2720-WGKHP,Male,1,No,No,2,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.00,181.70,0,2,Yes,0.990267,high
6473,5419-JPRRN,Male,0,No,No,1,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,101.45,101.45,0,1,Yes,0.989739,high
994,1374-DMZUI,Female,1,No,No,4,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.30,424.45,1,2,Yes,0.988538,high
4509,2012-NWRPA,Female,1,Yes,No,11,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.55,1131.20,0,1,Yes,0.987523,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,3779-OSWCF,Female,0,Yes,No,71,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),93.20,6506.15,0,0,No,0.000148,low
2692,6461-SZMCV,Female,0,Yes,No,71,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Bank transfer (automatic),87.95,6365.35,0,0,No,0.000148,low
28,5248-YGIJN,Male,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),90.25,6369.45,1,0,No,0.000143,low
5173,5727-MYATE,Male,0,Yes,Yes,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),90.80,6397.60,0,0,No,0.000000,low


In [73]:
## Summary of Tenure, MonthlyCharges, TotalCharges, numAdminTickets, numTechTickets

df_num_summ = df_6.groupby(["Churn_bin"],as_index=False).mean()
df_num_summ = df_num_summ[["Churn_bin","tenure","MonthlyCharges","TotalCharges","numAdminTickets","numTechTickets"]]
df_num_summ

/var/folders/hn/4swgrrzj36vd9rq1mqmr39j80000gn/T/ipykernel_3075/3169062328.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_num_summ = df_6.groupby(["Churn_bin"],as_index=False).mean()


,Churn_bin,tenure,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets
0,low,40.239941,60.331727,2711.601194,0.526614,0.147527
1,slight,14.454545,63.253283,1115.306229,0.533670,0.510101
2,medium,16.883659,74.309014,1462.787833,0.470693,1.055062
3,high,15.496296,86.136296,1494.105185,0.483333,1.407407


In [ ]:
## Customers with a low churn probability have a high tenure and high totalCharges. 
## They also have less number of tech tickets

In [75]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['Dependents'])
cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
cross_check


Dependents,No,Yes,Ratio
Churn_bin,,,
low,3046,1726,0.361693
slight,462,132,0.222222
medium,945,181,0.160746
high,480,60,0.111111


In [79]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['Contract'])
#cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
row_sum = cross_check.sum(axis=1)
df_perc = cross_check.divide(row_sum, axis=0).multiply(100)

# add percentages as new columns to the original dataframe
cross_check[['A_perc', 'B_perc', 'C_perc']] = df_perc
cross_check

Contract,Month-to-month,One year,Two year,A_perc,B_perc,C_perc
Churn_bin,,,,,,
low,1819,1307,1646,38.118189,27.388935,34.492875
slight,541,44,9,91.077441,7.407407,1.515152
medium,995,103,28,88.365897,9.147425,2.486679
high,520,18,2,96.296296,3.333333,0.370370


In [ ]:
## High and Medium churn customers have a higher percentage of customers in Month-to-month. 

In [95]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['InternetService'])
#cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
row_sum = cross_check.sum(axis=1)
df_perc = cross_check.divide(row_sum, axis=0).multiply(100)

# add percentages as new columns to the original dataframe
cross_check[['A_perc', 'B_perc', 'C_perc']] = df_perc
cross_check

InternetService,DSL,Fiber optic,No,A_perc,B_perc,C_perc
Churn_bin,,,,,,
low,1854,1497,1421,38.851635,31.370495,29.777871
slight,222,305,67,37.373737,51.346801,11.279461
medium,303,791,32,26.909414,70.248668,2.841918
high,37,503,0,6.851852,93.148148,0.000000


In [81]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['PaymentMethod'])
#cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
row_sum = cross_check.sum(axis=1)
df_perc = cross_check.divide(row_sum, axis=0).multiply(100)

# add percentages as new columns to the original dataframe
cross_check[['A_perc', 'B_perc', 'C_perc','D_perc']] = df_perc
cross_check

PaymentMethod,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check,A_perc,B_perc,C_perc,D_perc
Churn_bin,,,,,,,,
low,1263,1273,1000,1236,26.466890,26.676446,20.955574,25.901090
slight,82,78,279,155,13.804714,13.131313,46.969697,26.094276
medium,154,139,655,178,13.676732,12.344583,58.170515,15.808171
high,43,31,431,35,7.962963,5.740741,79.814815,6.481481


In [93]:

col_list = ['Churn_bin','PhoneService','MultipleLines',
            'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection',
            'TechSupport','StreamingTV','StreamingMovies']
df_cat_summ = df_6.groupby(col_list,as_index=False).size()
df_cat_summ = df_cat_summ[df_cat_summ['size']!=0]
df_cat_summ.sort_values('size',inplace=True,ascending=False)

with pd.option_context("display.max.columns",None):
    display(df_cat_summ)

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
8383,low,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,1087
12757,low,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,334
12392,low,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,110
33534,medium,Yes,No,Fiber optic,No,No,No,No,No,No,95
11663,low,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,87
...,...,...,...,...,...,...,...,...,...,...,...
37745,medium,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,1
37827,medium,Yes,Yes,DSL,Yes,Yes,No,No,No,No,1
37845,medium,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,1
37851,medium,Yes,Yes,DSL,Yes,Yes,No,Yes,Yes,No,1


In [94]:
df_cat_summ[df_cat_summ['Churn_bin']=='high']

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
46656,high,Yes,No,Fiber optic,No,No,No,No,No,No,58
51038,high,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,49
51092,high,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,38
46664,high,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,30
51030,high,Yes,Yes,Fiber optic,No,No,No,No,No,No,30
...,...,...,...,...,...,...,...,...,...,...,...
45981,high,Yes,No,DSL,No,No,Yes,No,No,No,1
46097,high,Yes,No,DSL,No,Yes,No,No,Yes,Yes,1
46109,high,Yes,No,DSL,No,Yes,No,Yes,No,Yes,1
46712,high,Yes,No,Fiber optic,No,No,Yes,No,No,Yes,1


In [96]:
df_cat_summ[df_cat_summ['Churn_bin']=='medium']

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
33534,medium,Yes,No,Fiber optic,No,No,No,No,No,No,95
37908,medium,Yes,Yes,Fiber optic,No,No,No,No,No,No,85
32805,medium,Yes,No,DSL,No,No,No,No,No,No,73
33540,medium,Yes,No,Fiber optic,No,No,No,No,Yes,No,37
37910,medium,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,37
...,...,...,...,...,...,...,...,...,...,...,...
37671,medium,Yes,Yes,DSL,Yes,No,No,No,Yes,No,1
37745,medium,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,1
37827,medium,Yes,Yes,DSL,Yes,Yes,No,No,No,No,1
37845,medium,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,1


In [97]:
df_cat_summ[df_cat_summ['Churn_bin']=='slight']

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
21505,slight,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,64
19683,slight,Yes,No,DSL,No,No,No,No,No,No,49
20412,slight,Yes,No,Fiber optic,No,No,No,No,No,No,47
24786,slight,Yes,Yes,Fiber optic,No,No,No,No,No,No,27
20169,slight,Yes,No,DSL,Yes,No,No,No,No,No,15
...,...,...,...,...,...,...,...,...,...,...,...
24621,slight,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,No,1
24615,slight,Yes,Yes,DSL,Yes,No,Yes,Yes,No,No,1
24561,slight,Yes,Yes,DSL,Yes,No,No,Yes,No,No,1
24293,slight,Yes,Yes,DSL,No,Yes,Yes,Yes,No,Yes,1


In [99]:

col_list = ['Churn_bin','PhoneService',
            'InternetService','StreamingTV','StreamingMovies']
df_cat_summ_2 = df_6.groupby(col_list,as_index=False).size()
df_cat_summ_2 = df_cat_summ_2[df_cat_summ_2['size']!=0]
df_cat_summ_2.sort_values(by=['Churn_bin','size'],inplace=True,ascending=False)

with pd.option_context("display.max.columns",None):
    display(df_cat_summ_2)

,Churn_bin,PhoneService,InternetService,StreamingTV,StreamingMovies,size
206,high,Yes,Fiber optic,Yes,Yes,248
198,high,Yes,Fiber optic,No,No,120
200,high,Yes,Fiber optic,No,Yes,72
204,high,Yes,Fiber optic,Yes,No,63
162,high,No,DSL,No,No,12
164,high,No,DSL,No,Yes,9
170,high,No,DSL,Yes,Yes,8
168,high,No,DSL,Yes,No,4
197,high,Yes,DSL,Yes,Yes,2
189,high,Yes,DSL,No,No,1


## EDA - Version 2

In [19]:
## Obtaining churn probabilities

df_5 = df_4.drop(["Churn_Yes"],axis=1)
pred_prob_overall = best_model.predict_proba(df_5)
df_5['Churn_Prob'] = pred_prob_overall[:,1]

In [20]:
path = '/Users/kiranjyothisheena/Documents/Kiran_Files/Cloud_computing/Project/'
df_6 = pd.read_excel(path + 'Telecom Churn Rate Dataset.xlsx')

df_6['TotalCharges'] = df_6['TotalCharges'].replace(' ',np.nan)
df_6 = df_6.dropna(how='any').reset_index(drop=True)

df_6['Churn_Prob'] = pred_prob_overall[:,1]
df_6.sort_values(by=["Churn_Prob"],inplace=True,ascending=False)
df_6

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn,Churn_Prob
994,1374-DMZUI,Female,1,No,No,4,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,94.30,424.45,1,2,Yes,0.988220
1699,0107-YHINA,Male,0,No,Yes,1,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,99.75,99.75,0,1,Yes,0.987897
966,3158-MOERK,Female,0,No,No,2,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,96.00,174.80,0,1,Yes,0.987731
1971,9497-QCMMS,Male,1,No,No,1,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,93.55,93.55,0,1,Yes,0.987372
842,3027-ZTDHO,Male,0,No,No,1,Yes,No,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,89.90,89.90,0,2,Yes,0.986069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,0971-QIFJK,Female,0,Yes,No,69,Yes,Yes,No,No internet service,...,No internet service,Two year,No,Bank transfer (automatic),24.25,1641.80,0,0,No,0.000412
3977,5884-FBCTL,Female,0,Yes,Yes,72,Yes,Yes,No,No internet service,...,No internet service,Two year,No,Bank transfer (automatic),25.10,1857.85,4,0,No,0.000378
844,6916-HIJSE,Female,0,No,No,65,Yes,No,DSL,Yes,...,Yes,Two year,Yes,Bank transfer (automatic),84.85,5459.20,0,0,No,0.000308
28,5248-YGIJN,Male,0,Yes,No,72,Yes,Yes,DSL,Yes,...,Yes,Two year,Yes,Credit card (automatic),90.25,6369.45,1,0,No,0.000279


In [21]:
## Creating buckets based on churn rate

df_6['Churn_bin'] = pd.cut(df_6['Churn_Prob'],
                           bins=[0, 0.5, 0.8, 1],
                           labels=['low', "medium", 'high'],
                          include_lowest=True)
                                  
with pd.option_context("display.max.columns",None):
    display(df_6)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn,Churn_Prob,Churn_bin
994,1374-DMZUI,Female,1,No,No,4,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.30,424.45,1,2,Yes,0.988220,high
1699,0107-YHINA,Male,0,No,Yes,1,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.75,99.75,0,1,Yes,0.987897,high
966,3158-MOERK,Female,0,No,No,2,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,96.00,174.80,0,1,Yes,0.987731,high
1971,9497-QCMMS,Male,1,No,No,1,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,93.55,93.55,0,1,Yes,0.987372,high
842,3027-ZTDHO,Male,0,No,No,1,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,89.90,89.90,0,2,Yes,0.986069,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,0971-QIFJK,Female,0,Yes,No,69,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),24.25,1641.80,0,0,No,0.000412,low
3977,5884-FBCTL,Female,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),25.10,1857.85,4,0,No,0.000378,low
844,6916-HIJSE,Female,0,No,No,65,Yes,No,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),84.85,5459.20,0,0,No,0.000308,low
28,5248-YGIJN,Male,0,Yes,No,72,Yes,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),90.25,6369.45,1,0,No,0.000279,low


In [22]:
## Summary of Tenure, MonthlyCharges, TotalCharges, numAdminTickets, numTechTickets

df_num_summ = df_6.groupby(["Churn_bin"],as_index=False).mean()
df_num_summ = df_num_summ[["Churn_bin","tenure","MonthlyCharges","TotalCharges","numAdminTickets","numTechTickets"]]
df_num_summ

/var/folders/hn/4swgrrzj36vd9rq1mqmr39j80000gn/T/ipykernel_926/3169062328.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_num_summ = df_6.groupby(["Churn_bin"],as_index=False).mean()


,Churn_bin,tenure,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets
0,low,40.434911,60.143671,2722.767075,0.528529,0.146450
1,medium,14.606171,66.362568,1161.609483,0.495463,0.571688
2,high,17.158598,81.744240,1579.249791,0.479132,1.362270


In [ ]:
## Customers with a low churn probability have a high tenure and high totalCharges. 
## They also have less number of tech tickets

In [105]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['Dependents'])
cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
cross_check


Dependents,No,Yes,Ratio
Churn_bin,,,
low,3046,1726,0.361693
medium,841,214,0.202844
high,1046,159,0.131950


In [24]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['Partner'])
cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
cross_check


Partner,No,Yes,Ratio
Churn_bin,,,
low,2106,2626,0.554945
medium,758,344,0.312160
high,775,423,0.353088


In [106]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['Contract'])
#cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
row_sum = cross_check.sum(axis=1)
df_perc = cross_check.divide(row_sum, axis=0).multiply(100)

# add percentages as new columns to the original dataframe
cross_check[['A_perc', 'B_perc', 'C_perc']] = df_perc
cross_check

Contract,Month-to-month,One year,Two year,A_perc,B_perc,C_perc
Churn_bin,,,,,,
low,1819,1307,1646,38.118189,27.388935,34.492875
medium,947,93,15,89.763033,8.815166,1.421801
high,1109,72,24,92.033195,5.975104,1.991701


In [ ]:
## High and Medium churn customers have a higher percentage of customers in Month-to-month. 

In [107]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['InternetService'])
#cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
row_sum = cross_check.sum(axis=1)
df_perc = cross_check.divide(row_sum, axis=0).multiply(100)

# add percentages as new columns to the original dataframe
cross_check[['A_perc', 'B_perc', 'C_perc']] = df_perc
cross_check

InternetService,DSL,Fiber optic,No,A_perc,B_perc,C_perc
Churn_bin,,,,,,
low,1854,1497,1421,38.851635,31.370495,29.777871
medium,375,590,90,35.545024,55.924171,8.530806
high,187,1009,9,15.518672,83.734440,0.746888


In [108]:
cross_check = pd.crosstab(df_6['Churn_bin'],df_6['PaymentMethod'])
#cross_check['Ratio'] = cross_check['Yes']/(cross_check['No'] + cross_check['Yes'])
row_sum = cross_check.sum(axis=1)
df_perc = cross_check.divide(row_sum, axis=0).multiply(100)

# add percentages as new columns to the original dataframe
cross_check[['A_perc', 'B_perc', 'C_perc','D_perc']] = df_perc
cross_check

PaymentMethod,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check,A_perc,B_perc,C_perc,D_perc
Churn_bin,,,,,,,,
low,1263,1273,1000,1236,26.466890,26.676446,20.955574,25.901090
medium,139,143,534,239,13.175355,13.554502,50.616114,22.654028
high,140,105,831,129,11.618257,8.713693,68.962656,10.705394


In [109]:

col_list = ['Churn_bin','PhoneService','MultipleLines',
            'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection',
            'TechSupport','StreamingTV','StreamingMovies']
df_cat_summ = df_6.groupby(col_list,as_index=False).size()
df_cat_summ = df_cat_summ[df_cat_summ['size']!=0]
df_cat_summ.sort_values('size',inplace=True,ascending=False)

with pd.option_context("display.max.columns",None):
    display(df_cat_summ)

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
8383,low,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,1087
12757,low,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,334
33534,high,Yes,No,Fiber optic,No,No,No,No,No,No,119
12392,low,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,110
19683,medium,Yes,No,DSL,No,No,No,No,No,No,95
...,...,...,...,...,...,...,...,...,...,...,...
28673,high,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,1
28917,high,No,No phone service,DSL,Yes,No,No,No,No,No,1
28935,high,No,No phone service,DSL,Yes,No,No,Yes,No,No,1
28941,high,No,No phone service,DSL,Yes,No,No,Yes,Yes,No,1


In [110]:
df_cat_summ[df_cat_summ['Churn_bin']=='high']

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
33534,high,Yes,No,Fiber optic,No,No,No,No,No,No,119
37908,high,Yes,Yes,Fiber optic,No,No,No,No,No,No,85
37916,high,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,70
38132,high,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,52
37970,high,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,52
...,...,...,...,...,...,...,...,...,...,...,...
28673,high,No,No phone service,DSL,No,Yes,Yes,Yes,Yes,Yes,1
28917,high,No,No phone service,DSL,Yes,No,No,No,No,No,1
28935,high,No,No phone service,DSL,Yes,No,No,Yes,No,No,1
28941,high,No,No phone service,DSL,Yes,No,No,Yes,Yes,No,1


In [111]:
df_cat_summ[df_cat_summ['Churn_bin']=='medium']

,Churn_bin,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,size
19683,medium,Yes,No,DSL,No,No,No,No,No,No,95
21505,medium,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,84
20412,medium,Yes,No,Fiber optic,No,No,No,No,No,No,81
24786,medium,Yes,Yes,Fiber optic,No,No,No,No,No,No,57
24788,medium,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,28
...,...,...,...,...,...,...,...,...,...,...,...
20225,medium,Yes,No,DSL,Yes,No,Yes,No,No,Yes,1
20177,medium,Yes,No,DSL,Yes,No,No,No,Yes,Yes,1
19871,medium,Yes,No,DSL,No,Yes,No,Yes,Yes,Yes,1
19865,medium,Yes,No,DSL,No,Yes,No,Yes,No,Yes,1


In [113]:

col_list = ['Churn_bin','PhoneService',
            'InternetService','StreamingTV','StreamingMovies']
df_cat_summ_2 = df_6.groupby(col_list,as_index=False).size()
df_cat_summ_2 = df_cat_summ_2[df_cat_summ_2['size']!=0]
df_cat_summ_2.sort_values(by=['Churn_bin','size'],inplace=True,ascending=False)

with pd.option_context("display.max.columns",None):
    display(df_cat_summ_2)

,Churn_bin,PhoneService,InternetService,StreamingTV,StreamingMovies,size
152,high,Yes,Fiber optic,Yes,Yes,413
144,high,Yes,Fiber optic,No,No,296
146,high,Yes,Fiber optic,No,Yes,150
150,high,Yes,Fiber optic,Yes,No,150
135,high,Yes,DSL,No,No,57
108,high,No,DSL,No,No,40
116,high,No,DSL,Yes,Yes,25
110,high,No,DSL,No,Yes,21
137,high,Yes,DSL,No,Yes,15
114,high,No,DSL,Yes,No,12


### Marketing Initiatives

**1) High (80 - 100)** <br>
a) Bundle customers seem to have high churn. Offer a single service with discounts or upgraded plan.<br>
b) Fiber Optic customers are unsatisifed. Offer DSL with low rates.<br>
c) Offer long-term renewal subscription with a guaranteed price per month <br>
d) Provide personal consultation to address issues for high values customers. 

**2) Medium (50 - 80)** <br>
a) Receive messaging and retention campaigns that increase engagement <br>
b) Offer a family plan to reduce costs

**3) Low (0 - 50)**<br>
a) Offer Premium service or additional services to increase revenue.

Ref:- a) https://www.mathereconomics.com/2020/06/11/segment-customers-by-churn-risk-price-elasticity-for-long-term-profitability/
b)

In [ ]:
explainer = shap.Explainer(model.predict, X_test)
shap_values = explainer.shap_values(X)

b) https://www.aidancooper.co.uk/a-non-technical-guide-to-interpreting-shap-analyses/